### Importação de bibliotecas

In [1]:

import geopandas as gpd
import pandas as pd

### Carragamento do GeoDataFrames das ruas

In [2]:
gdf_ruas = gpd.read_file('../dssg_pt/ruas_abril.geojson')


In [3]:
gdf_ruas.head(3)


,osm_id,rua,concelho,index_right,Freguesia,centroid_x,centroid_y,rua_freguesia_concelho,unique_index,geometry
0,4453413,Avenida da Liberdade,Marinha Grande,1174.0,Marinha Grande,-9.028758,39.757420,Avenida da Liberdade_Marinha Grande_Marinha Gr...,170,"LINESTRING (-9.02960 39.75781, -9.02954 39.757..."
1,4472168,Avenida da Liberdade,Marinha Grande,1174.0,Marinha Grande,-9.031063,39.758498,Avenida da Liberdade_Marinha Grande_Marinha Gr...,170,"LINESTRING (-9.03221 39.75903, -9.03204 39.758..."
2,5044364,Via 25 de Abril,Funchal,NaN,,-16.902049,32.652540,Via 25 de Abril__Funchal,1399,"LINESTRING (-16.90216 32.65268, -16.90207 32.6..."


### Carregamento GeoDataFrame de concelhos

In [4]:

gdf_concelhos = gpd.read_file('../dados_osm/concelhos-shapefile/concelhos.shp')
gdf_concelhos.head(3)


,ID_0,ISO,NAME_0,ID_1,NAME_1,ID_2,NAME_2,HASC_2,CCN_2,CCA_2,TYPE_2,ENGTYPE_2,NL_NAME_2,VARNAME_2,geometry
0,182,PRT,Portugal,1,Évora,1,Évora,PT.EV.EV,0,0705,Concelho,Municipality,None,None,"POLYGON ((-7.79291 38.76507, -7.79287 38.76506..."
1,182,PRT,Portugal,1,Évora,2,Alandroal,PT.EV.AL,0,0701,Concelho,Municipality,None,None,"POLYGON ((-7.25937 38.77351, -7.25921 38.77343..."
2,182,PRT,Portugal,1,Évora,3,Arraiolos,PT.EV.AR,0,0702,Concelho,Municipality,None,None,"POLYGON ((-7.88611 38.92495, -7.88580 38.92472..."


### Análise de coerência

Vejamos se existem concelhos que se repetem.

In [5]:
conc_a_analisar = gdf_concelhos.groupby(['NAME_2'])['NAME_2'].count().sort_values(ascending=False)
conc_a_analisar = list(conc_a_analisar.loc[conc_a_analisar.values > 1].index)

conc_a_analisar


['Braga', 'Lagoa', 'Calheta', 'Alcoutim']

Como se verifica existem 4 concelhos que e repetem. vamos analisar.

In [6]:
gdf_concelhos.loc[gdf_concelhos.NAME_2.isin(conc_a_analisar)].sort_values(['NAME_2'])

,ID_0,ISO,NAME_0,ID_1,NAME_1,ID_2,NAME_2,HASC_2,CCN_2,CCA_2,TYPE_2,ENGTYPE_2,NL_NAME_2,VARNAME_2,geometry
121,182,PRT,Portugal,9,Faro,121,Alcoutim,PT.FA.AC,0,0802,Concelho,Municipality,None,None,"POLYGON ((-7.56960 37.52726, -7.56915 37.52715..."
122,182,PRT,Portugal,9,Faro,121,Alcoutim,PT.FA.AC,0,0814,Concelho,Municipality,None,None,"MULTIPOLYGON (((-7.61847 37.11625, -7.61847 37..."
68,182,PRT,Portugal,5,Braga,69,Braga,PT.BR.BR,0,0303,Concelho,Municipality,None,None,"POLYGON ((-8.35704 41.61721, -8.35661 41.61719..."
69,182,PRT,Portugal,5,Braga,69,Braga,PT.BR.BR,0,0308,Concelho,Municipality,None,None,"POLYGON ((-8.31783 41.54539, -8.31739 41.54216..."
34,182,PRT,Portugal,3,Azores,35,Calheta,PT.AC.CA,0,4501,Concelho,Municipality,None,None,"MULTIPOLYGON (((-28.03560 38.60697, -28.03561 ..."
183,182,PRT,Portugal,13,Madeira,182,Calheta,PT.MA.CT,0,3101,Concelho,Municipality,None,None,"MULTIPOLYGON (((-17.24440 32.78358, -17.24445 ..."
37,182,PRT,Portugal,3,Azores,38,Lagoa,PT.AC.LG,0,4201,Concelho,Municipality,None,None,"POLYGON ((-25.55903 37.77534, -25.55883 37.775..."
126,182,PRT,Portugal,9,Faro,125,Lagoa,PT.FA.LA,0,0806,Concelho,Municipality,None,None,"MULTIPOLYGON (((-8.44033 37.08669, -8.44038 37..."


Por consulta aos códigos dos concelhos em https://info.portaldasfinancas.gov.pt/pt/apoio_contribuinte/Documents/ListaFreguesiasVigentes_SFs.xls, por concatenação dos campos `Código Distrito` e `Código Concelho`, conclui-se que:

    - 0814 - corresponde a Tavira
    - 0308 - corresponde a Guimarães

Relativamente a `Calheta` não se verifica anomalia pois existe concelhos com o mesmo nome nos Açores e na Madeira, o mesmo acontece com `Lagoa`, pois também temos concelhos com o mesmo nome nos Açores e no distrito de Faro.

Vamos então corrigir estas situações.

In [7]:
gdf_concelhos.loc[gdf_concelhos.CCA_2 == '0814', 'NAME_2'] = 'Tavira'
gdf_concelhos.loc[gdf_concelhos.CCA_2 == '0308', 'NAME_2'] = 'Guimarães'


In [8]:
gdf_concelhos = gdf_concelhos[['CCA_2', 'NAME_2', 'NAME_1', 'geometry']]
gdf_concelhos = gdf_concelhos.rename(columns={'CCA_2': 'id', 'NAME_2': 'concelho_ok', 'NAME_1': 'distrito'})
gdf_concelhos


,id,concelho_ok,distrito,geometry
0,0705,Évora,Évora,"POLYGON ((-7.79291 38.76507, -7.79287 38.76506..."
1,0701,Alandroal,Évora,"POLYGON ((-7.25937 38.77351, -7.25921 38.77343..."
2,0702,Arraiolos,Évora,"POLYGON ((-7.88611 38.92495, -7.88580 38.92472..."
3,0703,Borba,Évora,"POLYGON ((-7.46362 38.92344, -7.46344 38.92329..."
4,0704,Estremoz,Évora,"POLYGON ((-7.52770 39.00080, -7.52765 39.00066..."
...,...,...,...,...
303,1820,Tarouca,Viseu,"POLYGON ((-7.72591 41.08079, -7.72442 41.08028..."
304,1821,Tondela,Viseu,"POLYGON ((-8.12772 40.64028, -8.12751 40.64027..."
305,1822,Vila Nova de Paiva,Viseu,"POLYGON ((-7.77933 40.93439, -7.77835 40.93436..."
306,1823,Viseu,Viseu,"POLYGON ((-7.80686 40.83789, -7.80653 40.83787..."


### Merge Ruas com concelhos

Para não gerar conflito no join, vamos começar por remover a coluna `index_right`.

In [9]:
gdf_ruas = gdf_ruas.drop('index_right', axis=1)
gdf_ruas.head(2)


,osm_id,rua,concelho,Freguesia,centroid_x,centroid_y,rua_freguesia_concelho,unique_index,geometry
0,4453413,Avenida da Liberdade,Marinha Grande,Marinha Grande,-9.028758,39.757420,Avenida da Liberdade_Marinha Grande_Marinha Gr...,170,"LINESTRING (-9.02960 39.75781, -9.02954 39.757..."
1,4472168,Avenida da Liberdade,Marinha Grande,Marinha Grande,-9.031063,39.758498,Avenida da Liberdade_Marinha Grande_Marinha Gr...,170,"LINESTRING (-9.03221 39.75903, -9.03204 39.758..."


Efetuemos o join dos GeoDataFrames.

In [10]:
gdf_ruas = gpd.sjoin(gdf_ruas, gdf_concelhos)
gdf_ruas.head(3)

,osm_id,rua,concelho,Freguesia,centroid_x,centroid_y,rua_freguesia_concelho,unique_index,geometry,index_right,id,concelho_ok,distrito
0,4453413,Avenida da Liberdade,Marinha Grande,Marinha Grande,-9.028758,39.757420,Avenida da Liberdade_Marinha Grande_Marinha Gr...,170,"LINESTRING (-9.02960 39.75781, -9.02954 39.757...",160,1010,Marinha Grande,Leiria
1,4472168,Avenida da Liberdade,Marinha Grande,Marinha Grande,-9.031063,39.758498,Avenida da Liberdade_Marinha Grande_Marinha Gr...,170,"LINESTRING (-9.03221 39.75903, -9.03204 39.758...",160,1010,Marinha Grande,Leiria
726,118755649,Rua 25 de Abril,Marinha Grande,Vieira de Leiria,-8.970061,39.872512,Rua 25 de Abril_Vieira de Leiria_Marinha Grande,860,"LINESTRING (-8.96966 39.87237, -8.97019 39.872...",160,1010,Marinha Grande,Leiria


Curiosidade! Quantas ruas tem Tavira, alusivas ao 25 de abril?

In [11]:
gdf_ruas.loc[gdf_ruas.concelho_ok == 'Tavira']['unique_index'].nunique()

7

### Ruas por concelho alusivas ao 25 abril

In [12]:
df_qtd_ruas = pd.DataFrame(gdf_ruas.groupby(['concelho_ok'])['unique_index'].nunique()).reset_index()
df_qtd_ruas = df_qtd_ruas.rename(columns={'concelho_ok': 'concelho', 'unique_index': 'qtd_ruas'})
df_qtd_ruas


,concelho,qtd_ruas
0,Abrantes,7
1,Aguiar da Beira,2
2,Alandroal,2
3,Albergaria-a-Velha,3
4,Albufeira,8
...,...,...
230,Vizela,3
231,Águeda,6
232,Évora,19
233,Ílhavo,6


Criemos o ficheiro csv para exportação para o datawrapper.

In [13]:
df_qtd_ruas.to_csv('../outputs/ruas_por_concelho.csv', index = False, encoding='utf-8')
